# AI Text Assistant

🤖 **AI Writing Assistant** to generate text for you, evalaute the generated text and keep ehnacing it, developed based on LangChain, langgraph and gemini llm. The example is for generating attractive post titles about some topic of your choice.

By: [Ibrahim Sobh](https://www.linkedin.com/in/ibrahim-sobh-phd-8681757/)


## Install

In [ ]:
%pip install -U --quiet  langchain langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.2/806.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.4/252.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 5.8 MB/s eta 0:00:00


In [ ]:
!pip install --quiet langchain-google-genai

## LLM

In [ ]:
# Run this cell and paste the API key in the prompt
import os
import getpass
os.environ['GOOGLE_API_KEY'] = getpass.getpass('Gemini API Key:')

Gemini API Key:··········


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.7, top_p=0.85)

## Define Nodes

In [ ]:
from langchain import PromptTemplate

# generate
prompt = PromptTemplate.from_template("Tell me a very short title for a facebook post about {subject}.")
gen_chain = prompt | llm

In [ ]:
# Generate
generated_txt = gen_chain.invoke({"subject": "Software testing"})
generated_txt.content

'Software Testing: The Unsung Heroes of Quality'

In [ ]:
# Evaluate
prompt = PromptTemplate.from_template("Evaluate this title {title} to be one of: very good, good or bad")
eval_chain = prompt | llm

In [ ]:
txt_score = eval_chain.invoke({"title": generated_txt.content})
txt_score.content

'Very good'

In [ ]:
# Enhance
prompt = PromptTemplate.from_template("Rephrase this post title {title} to be a very good title one suitable for social media")
rewrite_chain = prompt | llm

In [ ]:
re_txt = rewrite_chain.invoke({"title": generated_txt.content})
re_txt.content

"**Testing: The Secret Weapon in Software's Success Story**"

In [ ]:
eval_chain.invoke({"title": generated_txt.content})

AIMessage(content='Very good')

## Graph

In [ ]:
import re
from typing import Dict, TypedDict
from langchain_core.messages import BaseMessage

class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        keys: A dictionary where each key is a string.
    """

    keys: Dict[str, any]

In [ ]:
def generate(state):
    """
    Generate txt

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """

    ## State
    state_dict = state["keys"]
    iter = state_dict["iterations"]
    subject = state_dict["subject"]

    res = gen_chain.invoke({"subject": subject})

    iter = iter+1

    return {"keys": {"generation": res.content, "subject":subject ,"iterations":iter}}

In [ ]:
def evaluate(state):
    """
    Evaluate txt

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    ## State
    state_dict = state["keys"]
    generated_solution = state_dict["generation"]
    iter = state_dict["iterations"]
    subject = state_dict["subject"]

    r = eval_chain.invoke({"title": generated_solution})

    if re.search('very good', r.content, re.IGNORECASE):
      error = "None"
    else:
      error = "FAIL"

    return {"keys": {"generation": generated_solution,
                     "error": error,
                     "subject": subject,
                     "iterations":iter}}

In [ ]:
def rephrase(state):
    """
    Rephrase txt

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """

    ## State
    state_dict = state["keys"]
    generated_solution = state_dict["generation"]
    iter = state_dict["iterations"]
    subject = state_dict["subject"]

    res = rewrite_chain.invoke({"title": generated_solution})
    re_txt.content

    iter = iter+1
    return {"keys": {"generation": res.content, "subject":subject ,"iterations":iter}}

In [ ]:
def decide_to_finish(state):
    """
    Determines whether to finish or re-try

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    state_dict = state["keys"]
    generated_solution = state_dict["generation"]
    error = state_dict["error"]
    iter = state_dict["iterations"]

    print(state)
    if error == "None":
        return "end"
    else:
        return "rephrase"

In [ ]:
from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("generate", generate)  # generation initial txt
workflow.add_node("evaluate", evaluate)  # evalaute txt
workflow.add_node("rephrase", rephrase)  # re-phrase txt


# Build graph
workflow.set_entry_point("generate")
workflow.add_edge("generate", "evaluate")
workflow.add_edge("rephrase", "evaluate")
workflow.add_conditional_edges(
    "evaluate",
    decide_to_finish,
    {
        "end": END,
        "rephrase": "rephrase",
    },
)

# Compile
app = workflow.compile()

In [ ]:
config = {"recursion_limit": 200}
fina_state = app.invoke({"keys":{"iterations":0, "subject":"software unit testing"}},config=config)

{'keys': {'generation': 'Unit Test: Protect Your Code', 'error': 'FAIL', 'subject': 'software unit testing', 'iterations': 1}}
{'keys': {'generation': '🛡️ Unit Test: The Ultimate Defense for Your Code 💪', 'error': 'None', 'subject': 'software unit testing', 'iterations': 2}}
